# Простейшая рекуррентная сеть
В этом ноутбуке мы пройдемся по основам работы с RNN. Сегодня займемся задачей генерации текста. 

In [ ]:
import warnings
from typing import Iterable, Tuple
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.distributions.categorical import Categorical

warnings.filterwarnings("ignore")

В качестве обучающего датасета возьмем набор из 120 тысяч анекдотов на русском языке. 
[Ссылка на данные](https://archive.org/download/120_tysyach_anekdotov) и [пост на хабре про тематическое моделирование](https://habr.com/ru/companies/otus/articles/723306/)

In [ ]:
with open(r"../additional_materials/anek_djvu.txt", "r", encoding="utf-8") as f:
    text = f.read()
text[118:500]

Мы не хотим моделировать все подряд, поэтому разобьем датасет на отдельные анекдоты.  

In [ ]:
def cut_data(text):
    return text.replace("\n\n", "").split("<|startoftext|>")[1:]

In [ ]:
cut_text = cut_data(text)

In [ ]:
cut_text[1:6]

Сделаем для начала самую простую модель с токенами на уровне символов. Это значит, что каждому символу в тексте ставится в соответствие некоторое число. Некоторые способы токенизации используют части слов или, наоборот, части бинарного представления текста.

In [ ]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}


Напишем функции для энкодинга и декодинга нашего текста. Они будут преобразовывать список символов в список чисел и обратно.

In [ ]:
def encode(sentence, vocab):
    return [vocab[sys] for sys in sentence] # List of ints 

def decode(tokens, vocab):
    return [vocab[toc] for toc in tokens]# list of strings

In [ ]:
encode(cut_text[0], char2int)

Просто представления символов в виде числа не подходят для обучения моделей. На выходе должны быть вероятности всех возможных токенов из словаря. Поэтому модели удобно учить с помощью энтропии. К тому же, токены часто преобразуют из исходного представления в эмбеддинги, которые также позволяют получить более удобное представление в высокоразмерном пространстве. 

В итоге векторы в модели выглядят следующим образом:
![alt_text](../additional_materials/images/char_rnn.jfif)

Задание: реализуйте метод, который преобразует батч в бинарное представление.

In [ ]:
def one_hot_encode(int_words: torch.Tensor, vocab_size: int) -> torch.Tensor:
    """
    Encodes a batch of sentences (integer indices) into binary one-hot representation.
    
    Args:
        int_words (torch.Tensor): Tensor of size (batch_size, seq_len) containing word indices.
        vocab_size (int): Size of the vocabulary.

    Returns:
        torch.Tensor: One-hot encoded tensor of size (batch_size, seq_len, vocab_size).
    """
    words_one_hot = torch.zeros(
        (int_words.numel(), vocab_size), dtype=torch.float32, device=int_words.device
    )
    words_one_hot[torch.arange(words_one_hot.shape[0]), int_words.flatten().long()] = 1.0
    words_one_hot = words_one_hot.reshape((*int_words.shape, vocab_size))
    return words_one_hot


Проверьте ваш код.

In [ ]:
test_seq = torch.tensor([[2, 6, 4, 1], [0,3, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

Однако, наши последовательности на самом деле разной длины. Как же объединить их в батч?

Реализуем два необходимых класса: 
- токенайзер, который будет брать текст, кодировать и декодировать символы. Еще одно, что будет реализовано там - добавлено несколько специальных символов (паддинг, конец последовательности, начало последовательности).
- Датасет, который будет брать набор шуток, используя токенайзер, строить эмбеддинги и дополнять последовательность до максимальной длины.

In [ ]:
class Tokenizer:
    def __init__(self, cut_text, max_len: int = 512):
        self.text = text
        self.max_len = max_len
        self.specials = ['<pad>', '<bos>', '<eos>']
        unique_chars = tuple(set(text))
        self.int2char = dict(enumerate(tuple(set(text))))
        self.char2int = {ch: ii for ii, ch in int2char.items()}
        self._add_special("<pad>")
        self._add_special('<bos>')
        self._add_special('<eos>')
    
    def _add_special(self, symbol) -> None:
        # add special characters to yuor dicts
        sym_num = len(self.char2int)
        self.char2int[symbol] = sym_num
        self.int2char[sym_num] = symbol

    @property
    def vocab_size(self):
        return len(self.int2char) # your code
        
    def decode_symbol(self, el):
        return self.int2char[el]
        
    def encode_symbol(self, el):
        return self.char2int[el]
        
    def str_to_idx(self, chars):
        return [self.char2int[sym] for sym in chars] # str -> list[int]

    def idx_to_str(self, idx):
        return [self.int2char[toc] for toc in idx] # list[int] -> list[str]

    def encode(self, chars):
        chars = ['<bos>'] + list(chars) + ['<eos>']
        return self.str_to_idx(chars)

    def decode(self, idx):
        chars = self.idx_to_str(idx)
        return "".join(chars) # make string from list

In [ ]:
class JokesDataset(Dataset):
    def __init__(self, tokenizer, cut_text, max_len: int = 512):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.cut_text = cut_text
        self.pad_index = self.tokenizer.encode("<pad>")
    
    def __len__(self):
        return len(self.cut_text)
    
    def __getitem__(self, idx):
        text = self.cut_text[idx]
        encoded = self.tokenizer.encode(text)
        encoded = encoded[:self.max_len]  # Ограничиваем длину
        input_sequence = torch.full((self.max_len,), self.pad_index, dtype=torch.long)
        target_sequence = torch.full((self.max_len,), self.pad_index, dtype=torch.long)
        
        # Заполняем входную и целевую последовательность
        input_sequence[:len(encoded) - 1] = torch.tensor(encoded[:-1])
        target_sequence[1:len(encoded)] = torch.tensor(encoded[1:])
        
        return input_sequence, target_sequence

In [ ]:
tokenizer = Tokenizer(text)
dataset = JokesDataset(tokenizer, cut_text, 512)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

Вопрос: А как бы мы должны были разделять данные на последовательности и батчи в случае, если бы использовался сплошной текст?

In [ ]:
for batch in dataloader:
    break
batch[1].shape

Теперь реализуем нашу модель. 
Необходимо следующее:
 - Используя токенайзер, задать размер словаря
 - Задать слой RNN с помощью torch.RNN. Доп.задание: создайте модель, используя слой LSTM.
 - Задать полносвязный слой с набором параметров: размерность ввода — n_hidden; размерность выхода — размер словаря. Этот слой преобразует состояние модели в логиты токенов.
 - Определить шаг forward, который будет использоваться при обучении
 - Определить метод init_hidden, который будет задавать начальное внутреннее состояние. Инициализировать будем нулями.
 - Определить метод inference, в котором будет происходить генерация последовательности из префикса. Здесь мы уже не используем явные логиты, а семплируем токены на их основе.


In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from typing import Tuple

class CharRNN(nn.Module):
    def __init__(
        self,
        tokenizer,
        hidden_dim: int = 256,
        num_layers: int = 2,
        drop_prob: float = 0.5,
        max_len: int = 512,
    ) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.drop_prob = drop_prob
        self.max_len = max_len
        
        # Токенизатор для кодирования и декодирования
        self.tokenizer = tokenizer
        self.vocab_size = tokenizer.vocab_size # размер словаря
        
        # RNN (или LSTM) слой
        self.rnn = nn.LSTM(
            input_size=self.vocab_size,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            dropout=self.drop_prob,
            batch_first=True,
        )
        
        # Dropout для регуляризации
        self.dropout = nn.Dropout(self.drop_prob)
        
        # Полносвязный слой: преобразует состояние RNN в логиты
        self.fc = nn.Linear(self.hidden_dim, self.vocab_size)

    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        # One-hot кодирование входной последовательности
        x = one_hot_encode(x, vocab_size=self.vocab_size)
        
        # Упаковка последовательностей для эффективности
        packed_embeds = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # Прогон через LSTM
        packed_outputs, hidden = self.rnn(packed_embeds)
        
        # Распаковка выхода обратно в тензор
        outputs, _ = pad_packed_sequence(packed_outputs, batch_first=True)
        
        # Dropout для регуляризации
        outputs = self.dropout(outputs)
        
        # Преобразование выхода RNN в логиты
        logits = self.fc(outputs)
        return logits, hidden

    def init_hidden(self, batch_size: int, device: str = "cpu") -> Tuple[torch.Tensor, torch.Tensor]:
        # Инициализация начального скрытого состояния нулями
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        return h0, c0

    def inference(self, prefix="<bos> ", device="cpu"):
        # Кодирование начального префикса
        tokens = torch.tensor(self.tokenizer.encode(prefix), dtype=torch.long, device=device).unsqueeze(0)
        
        # Создание one-hot представления
        inputs = one_hot_encode(tokens, vocab_size=self.vocab_size)
        
        # Инициализация скрытого состояния
        hidden = self.init_hidden(batch_size=1, device=device)
        
        # Генерация префикса
        outputs, hidden = self.rnn(inputs, hidden)
        logits = self.fc(outputs)
        
        # Семплирование токена
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        new_token = torch.multinomial(probs, num_samples=1)
        tokens = torch.cat([tokens, new_token], dim=1)
        
        # Остановка: достижение максимальной длины или EOS-токена
        while tokens.size(1) < self.max_len and new_token.item() != self.tokenizer.encode('<eos>'):
            inputs = one_hot_encode(new_token, vocab_size=self.vocab_size)
            outputs, hidden = self.rnn(inputs, hidden)
            logits = self.fc(outputs)
            probs = torch.softmax(logits[:, -1, :], dim=-1)
            new_token = torch.multinomial(probs, num_samples=1)
            tokens = torch.cat([tokens, new_token], dim=1)
        
        # Декодирование в строку
        return self.tokenizer.decode(tokens.squeeze().tolist())

Зададим параметры для обучения. Можете варьировать их, чтобы вам хватило ресурсов.

In [ ]:
batch_size = 16
seq_length = 512
n_hidden = 128
n_layers = 6
drop_prob = 0.1
lr = 0.1

Напишите функцию для одного тренировочного шага. В этом ноутбуке сам процесс обучения модели достаточно тривиален, поэтому мы не будем использовать сложные функции для обучающего цикла. Вы же, однако, можете дописать их.

In [ ]:
def training_step(
    model: CharRNN,
    train_batch: Tuple[torch.Tensor, torch.Tensor],
    vocab_size: int,
    criterion: nn.Module,
    optimizer,
    device="cpu"
) -> torch.Tensor:
    # Обнуляем градиенты
    optimizer.zero_grad()

    # Извлекаем данные из пакета
    inputs, targets = train_batch
    batch_size, seq_len = inputs.shape

    # Переносим данные на нужное устройство (например, GPU)
    inputs, targets = inputs.to(device), targets.to(device)

    # Прямой проход через модель
    lengths = (inputs != 0).sum(dim=1)  # или другая логика для определения длин
    logits, _ = model(inputs, lengths)  # Получаем логиты от модели

    # Переходим от логитов к потере
    # targets нужно сдвигать на 1, чтобы правильно сравнить предсказания и настоящие метки
    loss = criterion(logits.view(-1, vocab_size), targets.view(-1))

    # Обратный проход
    loss.backward()

    # Обновление весов
    optimizer.step()

    return loss


Инициализируйте модель, функцию потерь и оптимизатор.

In [ ]:
model = CharRNN(tokenizer, n_hidden, n_layers, drop_prob).to('cuda')
hidden = None
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


Проверьте необученную модель: она должна выдавать бессмысленные последовательности

In [ ]:
model.eval()  # Переключаем модель в режим оценки (inference)

# Шаг 3: Генерация текста
prefix = "<bos> "  # Начальный токен последовательности
generated_sequence = model.inference(prefix=prefix, device="cuda")

# Шаг 4: Вывод результата
print("Сгенерированная последовательность необученной моделью:")
print(generated_sequence)


In [ ]:
def plot_losses(losses):
    clear_output()
    plt.plot(range(1, len(losses) + 1), losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

Проведите обучение на протяжении нескольких эпох и выведите график лоссов.

In [ ]:
for train_batch in dataloader:
    inputs, targets = train_batch
    print(f"Inputs shape: {inputs.shape}")
    print(f"Targets shape: {targets.shape}")
    print(targets)
    break

In [ ]:
for batch_idx, train_batch in enumerate(dataloader):
    inputs, targets = train_batch
    print(f"Batch {batch_idx}: Inputs shape = {inputs.shape}, Targets shape = {targets.shape}")
    break

In [ ]:
losses = []
num_epochs = 5

# Основной цикл обучения
for epoch in range(1, num_epochs + 1):
    epoch_loss = 0.0  # Суммарные потери за эпоху
    model.train()  # Переключение в режим тренировки

    for batch_idx, train_batch in enumerate(dataloader):  # train_loader — DataLoader с батчами
        loss = training_step(model, train_batch, tokenizer.vocab_size, criterion, optimizer, device='cuda')
        losses.append(loss.item())  # Запись потерь
        epoch_loss += loss.item()

        # Логгирование каждые 100 батчей
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{batch_idx + 1}], Loss: {loss.item():.4f}")

    # Сохранение весов после каждой эпохи
    torch.save(model.state_dict(), f"rnn_epoch_{epoch}.pt")
    print(f"Epoch {epoch} completed. Average Loss: {epoch_loss / len(dataloader):.4f}")

    # Визуализация потерь
    plot_losses(losses)

# Финальное сохранение модели
torch.save(model.state_dict(), "rnn_final.pt")
print("Training completed and model saved.")

In [ ]:
model = CharRNN(tokenizer, n_hidden, n_layers, drop_prob).to('cuda')
model.load_state_dict(torch.load('rnn_epoch_1.pt'))

In [ ]:
[model.inference("<bos>", device='cuda') for _ in range(10)]

In [ ]:
import pymorphy2

class Tokenizer:
    def __init__(self, text, max_len: int = 512):
        """Инициализация токенизатора."""
        self.text = text
        self.max_len = max_len
        self.specials = ['<pad>', '<bos>', '<eos>']

        # Уникальные символы
        unique_chars = set(text)
        self.int2char = dict(enumerate(unique_chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        # Добавление специальных символов
        for special in self.specials:
            self._add_special(special)

        # Инициализация лемматизатора
        self.morph = pymorphy2.MorphAnalyzer()

    def _add_special(self, symbol: str) -> None:
        """Добавить специальный символ в словари."""
        if symbol not in self.char2int:
            sym_num = len(self.char2int)
            self.char2int[symbol] = sym_num
            self.int2char[sym_num] = symbol

    @property
    def vocab_size(self):
        """Возвращает размер словаря."""
        return len(self.int2char)

    def decode_symbol(self, el: int) -> str:
        """Декодирует индекс в символ."""
        return self.int2char.get(el, '<unk>')  # Возвращает '<unk>', если индекс отсутствует

    def encode_symbol(self, el: str) -> int:
        """Кодирует символ в индекс."""
        return self.char2int.get(el, self.char2int['<pad>'])  # Если символ отсутствует, подставляет '<pad>'

    def str_to_idx(self, chars: str) -> list:
        """Преобразует строку в список индексов."""
        return [self.char2int.get(sym, self.char2int['<pad>']) for sym in chars]

    def idx_to_str(self, idx: list) -> str:
        """Преобразует список индексов в строку."""
        return "".join([self.int2char.get(toc, '<unk>') for toc in idx])

    def encode(self, text: str) -> list:
        """Закодировать строку с учетом лемматизации."""
        lemmatized_text = self.lemmatize(text)
        chars = ['<bos>'] + list(lemmatized_text[:self.max_len - 2]) + ['<eos>']
        return self.str_to_idx(chars)

    def decode(self, idx: list) -> str:
        """Декодировать список индексов в строку."""
        chars = self.idx_to_str(idx)
        return "".join(chars).replace('<bos>', '').replace('<eos>', '').strip()

    def lemmatize(self, text: str) -> str:
        """Лемматизировать текст."""
        words = text.split()
        lemmatized = " ".join([self.morph.parse(word)[0].normal_form for word in words])
        return lemmatized

    def pad_sequence(self, sequence: list) -> list:
        """Дополняет или обрезает последовательность до нужной длины."""
        if len(sequence) < self.max_len:
            sequence += [self.char2int['<pad>']] * (self.max_len - len(sequence))
        return sequence[:self.max_len]


In [ ]:
tokenizer = Tokenizer(cut_text)

In [ ]:
print("<pad>:", tokenizer.char2int.get("<pad>"))
print("<bos>:", tokenizer.char2int.get("<bos>"))
print("<eos>:", tokenizer.char2int.get("<eos>"))
# Должно вывести индексы для специальных символов


In [ ]:
model = CharRNN(tokenizer, n_hidden, n_layers, drop_prob).to('cuda')
hidden = None
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
dataset = JokesDataset(tokenizer, cut_text, 512)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
losses = []
num_epochs = 5

# Основной цикл обучения
for epoch in range(1, num_epochs + 1):
    epoch_loss = 0.0  # Суммарные потери за эпоху
    model.train()  # Переключение в режим тренировки

    for batch_idx, train_batch in enumerate(dataloader):  # train_loader — DataLoader с батчами
        loss = training_step(model, train_batch, tokenizer.vocab_size, criterion, optimizer, device='cuda')
        losses.append(loss.item())  # Запись потерь
        epoch_loss += loss.item()

        # Логгирование каждые 100 батчей
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{batch_idx + 1}], Loss: {loss.item():.4f}")

    # Сохранение весов после каждой эпохи
    torch.save(model.state_dict(), f"rnn_epoch_{epoch}.pt")
    print(f"Epoch {epoch} completed. Average Loss: {epoch_loss / len(dataloader):.4f}")

    # Визуализация потерь
    plot_losses(losses)

# Финальное сохранение модели
torch.save(model.state_dict(), "rnn_final.pt")
print("Training completed and model saved.")

In [ ]:
torch.save(model.state_dict(), f"rnn_bert_epoch_{1}.pt")

In [ ]:
prefix = "<bos> "
tokens = torch.tensor(model.tokenizer.encode(prefix), dtype=torch.long, device='cuda').unsqueeze(0)
        
# Создание one-hot представления
inputs = one_hot_encode(tokens, vocab_size=model.vocab_size)

# Инициализация скрытого состояния
hidden = model.init_hidden(batch_size=1, device='cuda')

# Генерация префикса
outputs, hidden = model.rnn(inputs, hidden)
logits = model.fc(outputs)

# Семплирование токена
probs = torch.softmax(logits[:, -1, :], dim=-1)
new_token = torch.multinomial(probs, num_samples=1)
tokens = torch.cat([tokens, new_token], dim=1)

# Остановка: достижение максимальной длины или EOS-токена
while tokens.size(1) < model.max_len and new_token.item() != model.tokenizer.encode('<eos>'):
    inputs = one_hot_encode(new_token, vocab_size=model.vocab_size)
    outputs, hidden = model.rnn(inputs, hidden)
    logits = model.fc(outputs)
    probs = torch.softmax(logits[:, -1, :], dim=-1)
    new_token = torch.multinomial(probs, num_samples=1)
    tokens = torch.cat([tokens, new_token], dim=1)

# Декодирование в строку
model.tokenizer.decode(tokens.squeeze().tolist())

Теперь попробуем написать свой собственный RNN. Это будет довольно простая модель с одним слоем.


In [ ]:
# YOUR CODE: custom model nn.Module, changed CharRNN, etc